In [69]:
import pandas as pd
import numpy as nps

In [70]:
apart_list_df = pd.read_csv('output.csv')

In [71]:
train_df = pd.read_csv('../../data/train.csv')

/tmp/ipykernel_323528/529233478.py:1: DtypeWarning: Columns (16,17,36) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv('../../data/train.csv')


In [72]:
concat["아파트명"].isnull().sum()

2136

In [73]:
concat.isnull().sum()

시군구                             0
번지                            227
본번                             75
부번                             75
아파트명                         2136
전용면적(㎡)                         0
계약년월                            0
계약일                             0
층                               0
건축년도                            0
도로명                             0
해제사유발생일                   1121899
등기신청일자                          0
거래유형                            0
중개사소재지                          0
k-단지분류(아파트,주상복합등등)         877273
k-전화번호                     876850
k-팩스번호                     879348
단지소개기존clob                1058958
k-세대타입(분양형태)               876125
k-관리방식                     876125
k-복도유형                     876454
k-난방방식                     876125
k-전체동수                     877207
k-전체세대수                    876125
k-건설사(시공사)                 877637
k-시행사                      877834
k-사용검사일-사용승인일              876259
k-연면적                      876125
k-주거전용면적      

In [74]:
train_df.head(1)

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,124000


In [75]:
test_df = pd.read_csv('../../data/test.csv')

In [76]:
concat = pd.concat([test_df, train_df])

In [77]:
apart_list_df['kaptName'] = apart_list_df['kaptName'].str.replace('아파트', '', regex=False).str.strip()

In [91]:
df2_unique = apart_list_df.drop_duplicates(subset=['kaptName'])


In [92]:
concat_with_apart_code = pd.merge(concat, df2_unique, left_on='아파트명', right_on='kaptName', how='left')


In [93]:
concat_with_apart_code["아파트명"].isnull().sum()

2136

In [94]:
concat.shape

(1128094, 52)

In [95]:
concat_with_apart_code.shape

(1128094, 58)

In [97]:
concat_with_apart_code.isnull().sum()

시군구                             0
번지                            227
본번                             75
부번                             75
아파트명                         2136
전용면적(㎡)                         0
계약년월                            0
계약일                             0
층                               0
건축년도                            0
도로명                             0
해제사유발생일                   1121899
등기신청일자                          0
거래유형                            0
중개사소재지                          0
k-단지분류(아파트,주상복합등등)         877273
k-전화번호                     876850
k-팩스번호                     879348
단지소개기존clob                1058958
k-세대타입(분양형태)               876125
k-관리방식                     876125
k-복도유형                     876454
k-난방방식                     876125
k-전체동수                     877207
k-전체세대수                    876125
k-건설사(시공사)                 877637
k-시행사                      877834
k-사용검사일-사용승인일              876259
k-연면적                      876125
k-주거전용면적      

# 먼저 아파트별로 모으기 
- 요청 횟수를 줄이기 위함

In [103]:
unique_apartment_names = concat_with_apart_code['kaptCode'].unique()
unique_apartment_names = list(unique_apartment_names)

In [104]:
import requests

SERVICE_KEY = "u30edgoNbZopO75iWHAY10dSZyqctg6i7JyM8CuTnFzWmAaFLa8e4touRRvXPc%2FQJe%2FzS1cnWntXSSlfo6MVXQ%3D%3D"
import xml.etree.ElementTree as ET


In [105]:
all_data = []

# 아파트 코드 목록을 반복하여 XML 데이터 요청 및 파싱
for code in unique_apartment_names:
    url = f"http://apis.data.go.kr/1613000/AptBasisInfoService1/getAphusBassInfo?kaptCode={code}&ServiceKey={SERVICE_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        xml_data = response.text
    else:
        raise Exception(f"Failed to fetch data for code {code}: {response.status_code}")

    # XML 데이터 파싱
    root = ET.fromstring(xml_data)

    # item 요소 찾기
    item = root.find('.//item')
    if item is not None:
        # 데이터를 딕셔너리로 변환
        data = {child.tag: child.text for child in item}
        # 데이터를 리스트에 추가
        all_data.append(data)
    else:
        print(f"No data found for code {code}")

# 모든 데이터를 데이터프레임으로 변환
df = pd.DataFrame(all_data)

# 결과 출력
print(df)

No data found for code nan
        bjdCode codeAptNm codeHallNm codeHeatNm codeMgrNm codeSaleNm  \
0    1168010300       아파트        계단식       개별난방      자치관리         분양   
1    1168010300       아파트        계단식       지역난방      위탁관리         분양   
2    1168010300       아파트        혼합식       지역난방      자치관리         분양   
3    1168010300       아파트        복도식       지역난방      위탁관리         분양   
4    1168010300       아파트        혼합식       지역난방      위탁관리         분양   
..          ...       ...        ...        ...       ...        ...   
908  1159010900       NaN        계단식       개별난방      위탁관리         분양   
909  1141012000      주상복합        계단식       지역난방      위탁관리         분양   
910  1121510300       아파트        계단식       개별난방      위탁관리         분양   
911  1135010300       아파트        복도식       지역난방      위탁관리         임대   
912  1114016700       NaN        혼합식       중앙난방      위탁관리         분양   

               doroJuso     kaptAcompany  \
0       서울특별시 강남구 언주로 3               모름   
1     서울특별시 강남구 개포로 

In [106]:
df.to_csv("detailed.csv")

In [108]:
df.columns

Index(['bjdCode', 'codeAptNm', 'codeHallNm', 'codeHeatNm', 'codeMgrNm',
       'codeSaleNm', 'doroJuso', 'kaptAcompany', 'kaptAddr', 'kaptBaseFloor',
       'kaptBcompany', 'kaptCode', 'kaptDongCnt', 'kaptFax', 'kaptMarea',
       'kaptMparea_135', 'kaptMparea_136', 'kaptMparea_60', 'kaptMparea_85',
       'kaptName', 'kaptTarea', 'kaptTel', 'kaptTopFloor', 'kaptUrl',
       'kaptUsedate', 'kaptdaCnt', 'ktownFlrNo', 'privArea', 'zipcode',
       'kaptdEcntp'],
      dtype='object')

In [118]:
all_standard_data = []
# # 아파트 코드 목록을 반복하여 XML 데이터 요청 및 파싱
for code in unique_apartment_names:
    url = f"http://apis.data.go.kr/1613000/AptBasisInfoService1/getAphusBassInfo?kaptCode={code}&ServiceKey={SERVICE_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        xml_data = response.text
    else:
        raise Exception(f"Failed to fetch data for code {code}: {response.status_code}")

    # XML 데이터 파싱
    root = ET.fromstring(xml_data)

    # item 요소 찾기
    item = root.find('.//item')
    if item is not None:
        # 데이터를 딕셔너리로 변환
        data = {child.tag: child.text for child in item}
        # 데이터를 리스트에 추가
        all_standard_data.append(data)
    else:
        print(f"No data found for code {code}")

# 모든 데이터를 데이터프레임으로 변환
all_standard_data_Df = pd.DataFrame(all_standard_data)
all_standard_data_Df


No data found for code nan


,bjdCode,codeAptNm,codeHallNm,codeHeatNm,codeMgrNm,codeSaleNm,doroJuso,kaptAcompany,kaptAddr,kaptBaseFloor,...,kaptTarea,kaptTel,kaptTopFloor,kaptUrl,kaptUsedate,kaptdaCnt,ktownFlrNo,privArea,zipcode,kaptdEcntp
0,1168010300,아파트,계단식,개별난방,자치관리,분양,서울특별시 강남구 언주로 3,모름,서울특별시 강남구 개포동 658-1 개포6차우성아파트,0,...,22637.1,025776611,5,,19871121,270,5,20203.8,06316,NaN
1,1168010300,아파트,계단식,지역난방,위탁관리,분양,서울특별시 강남구 개포로 311,개포우성9차모델링조합,서울특별시 강남구 개포동 651-1 개포더샵트리에,3,...,44951.1774,025763999,16,thesharp-trieh.hthomeservice.com/#/feeManage?_...,20211201,232,16,24877.8335,06318,9
2,1168010300,아파트,혼합식,지역난방,자치관리,분양,서울특별시 강남구 개포로 307,우성건설,서울특별시 강남구 개포동 652 개포우성3차,1,...,63304.19,025749116,15,,19841231,405,15,53437.05,06319,11
3,1168010300,아파트,복도식,지역난방,위탁관리,분양,서울특별시 강남구 삼성로4길 17,대우건설,서울특별시 강남구 개포동 187 개포주공5단지,1,...,93810,024458879,14,,19831026,940,14,65959.9,06329,12
4,1168010300,아파트,혼합식,지역난방,위탁관리,분양,서울특별시 강남구 개포로 516,대한주택공사,서울특별시 강남구 개포동 185 개포주공6단지,1,...,94816.956,024458907,15,,19831026,1060,15,73333.1,06330,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,1159010900,NaN,계단식,개별난방,위탁관리,분양,NaN,(주)청민건설,서울특별시 동작구 신대방동 686-48 동작협성휴포레시그니처아파트,6,...,47065.0837,028498834,29,,20220527,274,29,23281.4591,07074,6
909,1141012000,주상복합,계단식,지역난방,위탁관리,분양,NaN,모래내 서중양대시장 정비사업,서울특별시 서대문구 남가좌동 286-1 DMC금호리첸시아,4,...,72268.03,023099745,29,,20220720,450,29,32941.15,03712,10
910,1121510300,아파트,계단식,개별난방,위탁관리,분양,서울특별시 광진구 아차산로 431,한국자산신탁(주),서울특별시 광진구 구의동 667 강변에스케이뷰,3,...,50015.09,024570728,29,,20170714,197,29,16723.47,05045,4
911,1135010300,아파트,복도식,지역난방,위탁관리,임대,서울특별시 노원구 동일로 1127,sh공사,서울특별시 노원구 공릉동 708 공릉1단지,0,...,60621.13,029748290,15,http://www.i-sh.co.kr/happy/intro.do,19941207,1395,15,37676.55,01851,18


In [119]:
all_standard_data_Df.to_csv("all_standard_data.csv")

In [120]:
all_standard_data_Df.columns

Index(['bjdCode', 'codeAptNm', 'codeHallNm', 'codeHeatNm', 'codeMgrNm',
       'codeSaleNm', 'doroJuso', 'kaptAcompany', 'kaptAddr', 'kaptBaseFloor',
       'kaptBcompany', 'kaptCode', 'kaptDongCnt', 'kaptFax', 'kaptMarea',
       'kaptMparea_135', 'kaptMparea_136', 'kaptMparea_60', 'kaptMparea_85',
       'kaptName', 'kaptTarea', 'kaptTel', 'kaptTopFloor', 'kaptUrl',
       'kaptUsedate', 'kaptdaCnt', 'ktownFlrNo', 'privArea', 'zipcode',
       'kaptdEcntp'],
      dtype='object')

In [125]:
merge_apart_code = pd.merge(concat_with_apart_code, all_standard_data_Df, on='kaptCode', how='left')

In [126]:
merge_apart_code.head(1)

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,kaptTarea,kaptTel,kaptTopFloor,kaptUrl,kaptUsedate,kaptdaCnt,ktownFlrNo,privArea,zipcode,kaptdEcntp
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,202307,26,5,1987,...,22637.1,025776611,5,,19871121,270,5,20203.8,06316,NaN


In [127]:
merge_apart_code.to_csv("before.csv")

In [130]:
merge_apart_code.columns

Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도',
       '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)',
       'k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일',
       'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-홈페이지',
       'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
       '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드',
       '좌표X', '좌표Y', '단지신청일', 'target', 'as1', 'as2', 'as3', 'bjdCode_x',
       'kaptCode', 'kaptName_x', 'bjdCode_y', 'codeAptNm', 'codeHallNm',
       'codeHeatNm', 'codeMgrNm', 'codeSaleNm', 'doroJuso', 'kaptAcompany',
       'kaptAddr', 'kaptBaseFloor', 'kaptBcompany', 'kaptDongCnt', 'kaptFax',
       'kaptMarea', 'kaptMparea_135', 'kaptMparea_136', 'kaptMparea_60',
       'kaptMparea_85', 'kaptName_

In [131]:
merge_apart_code['k-전체동수'] = merge_apart_code.apply(lambda row: row['kaptDongCnt'] if pd.isna(row['k-전체동수']) else row['k-전체동수'], axis=1)


In [134]:
equal_list = [("k-세대타입(분양형태)" ,'codeAptNm'), 
              ('k-난방방식', "codeHeatNm"),
                ("k-연면적","kaptTarea"),
                 ('k-전체세대수',"kaptdaCnt"),
                  ('k-복도유형', "codeHallNm"),
                   ('k-주거전용면적', "privArea") ]

for i, j in equal_list:
  merge_apart_code[i] = merge_apart_code.apply(lambda row: row[j] if pd.isna(row[i]) else row[j], axis=1)



In [137]:
columns_to_drop = [j for i, j in equal_list]
aprt = merge_apart_code.drop(columns=columns_to_drop)

In [141]:

to_korean= [("hoCnt", "전체호수"),('kaptTopFloor',"최상층")]
rename_dict = dict(to_korean)

df_renamed = aprt.rename(columns=rename_dict)


In [144]:
df_renamed.head(5)

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,kaptMparea_60,kaptMparea_85,kaptName_y,kaptTel,최상층,kaptUrl,kaptUsedate,ktownFlrNo,zipcode,kaptdEcntp
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.9700,202307,26,5,1987,...,20,250,개포6차우성아파트,025776611,5,,19871121,5,06316,NaN
1,서울특별시 강남구 개포동,651-1,651.0,1.0,개포더샵트리에,108.2017,202308,15,10,2021,...,0,0,개포더샵트리에,025763999,16,thesharp-trieh.hthomeservice.com/#/feeManage?_...,20211201,16,06318,9
2,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,161.0000,202307,28,15,1984,...,0,0,개포우성3차,025749116,15,,19841231,15,06319,11
3,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,133.4600,202308,10,14,1984,...,0,0,개포우성3차,025749116,15,,19841231,15,06319,11
4,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,104.4300,202308,18,6,1984,...,0,0,개포우성3차,025749116,15,,19841231,15,06319,11


In [145]:
df_renamed.columns

Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도',
       '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)',
       'k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일',
       'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-홈페이지',
       'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
       '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드',
       '좌표X', '좌표Y', '단지신청일', 'target', 'as1', 'as2', 'as3', 'bjdCode_x',
       'kaptCode', 'kaptName_x', 'bjdCode_y', 'codeMgrNm', 'codeSaleNm',
       'doroJuso', 'kaptAcompany', 'kaptAddr', 'kaptBaseFloor', 'kaptBcompany',
       'kaptDongCnt', 'kaptFax', 'kaptMarea', 'kaptMparea_135',
       'kaptMparea_136', 'kaptMparea_60', 'kaptMparea_85', 'kaptName_y',
       'kaptTel', '최상층', 'kaptUrl', 'kaptUse

In [148]:
merge_apart_code = df_renamed.drop(['as1', 'as2', 'as3', 'bjdCode_x',
       'kaptCode', 'kaptName_x', 'bjdCode_y', 'codeMgrNm', 'codeSaleNm',
       'doroJuso', 'kaptAcompany', 'kaptAddr', 'kaptBaseFloor', 'kaptBcompany',
        'kaptFax', 'kaptMarea', 'kaptMparea_135',
       'kaptMparea_136', 'kaptMparea_60', 'kaptMparea_85', 'kaptName_y',
       'kaptTel', 'kaptUrl', 'kaptUsedate', 'ktownFlrNo', 'zipcode',
       'kaptdEcntp'],axis=1)

In [150]:
merge_apart_code=merge_apart_code.drop(["kaptDongCnt"],axis=1)

In [151]:
merge_apart_code.head(2)

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target,최상층
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.9700,202307,26,5,1987,...,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0,NaN,5
1,서울특별시 강남구 개포동,651-1,651.0,1.0,개포더샵트리에,108.2017,202308,15,10,2021,...,305.0,의무,2022-02-23 13:01:10.0,Y,N,127.056394,37.484892,2022-02-23 11:05:05.0,NaN,16


In [152]:
merge_apart_code.shape[0]

1128094

In [153]:
merge_apart_code.to_csv('final.csv')